# FEATURE ENGINEERING
Feature engineering involves taking raw data an extracting feature from it that are suitable for tasks like Machine learning.

### GETTING TO KNOW YOUR DATA

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('survey.csv')
df.head(5)

,SurveyDate,FormalEducation,ConvertedSalary,Hobby,Country,StackOverflowJobsRecommend,VersionControl,Age,Years Experience,Gender,RawSalary
0,2/28/18 20:20,Bachelor's degree (BA. BS. B.Eng.. etc.),NaN,Yes,South Africa,NaN,Git,21,13,Male,NaN
1,6/28/18 13:26,Bachelor's degree (BA. BS. B.Eng.. etc.),70841.0,Yes,Sweeden,7.0,Git;Subversion,38,9,Male,"70,841.00"
2,6/6/18 3:37,Bachelor's degree (BA. BS. B.Eng.. etc.),NaN,No,Sweeden,8.0,Git,45,11,NaN,NaN
3,5/9/18 1:06,Some college/university study without earning ...,21426.0,Yes,Sweeden,NaN,Zip file back-ups,46,12,Male,"21,426.00"
4,4/12/18 22:41,Bachelor's degree (BA. BS. B.Eng.. etc.),41671.0,Yes,UK,8.0,Git,39,7,Male,"£41,671.00"


In [3]:
df.dtypes


SurveyDate                     object
FormalEducation                object
ConvertedSalary               float64
Hobby                          object
Country                        object
StackOverflowJobsRecommend    float64
VersionControl                 object
Age                             int64
Years Experience                int64
Gender                         object
RawSalary                      object
dtype: object

In [9]:
df_numeric = df.select_dtypes(include=['int64', 'float'])
print(df_numeric.columns)

Index(['ConvertedSalary', 'StackOverflowJobsRecommend', 'Age',
       'Years Experience'],
      dtype='object')


In [11]:
#One-hot encoding
#Convert the Country column to a one hot encoded Data Frame
one_hot_encoded = pd.get_dummies(df, columns=['Country'], prefix='OH')

In [12]:
print(one_hot_encoded.columns)

Index(['SurveyDate', 'FormalEducation', 'ConvertedSalary', 'Hobby',
       'StackOverflowJobsRecommend', 'VersionControl', 'Age',
       'Years Experience', 'Gender', 'RawSalary', 'OH_France', 'OH_India',
       'OH_Ireland', 'OH_Russia', 'OH_South Africa', 'OH_Spain', 'OH_Sweeden',
       'OH_UK', 'OH_USA', 'OH_Ukraine'],
      dtype='object')


In [13]:
#Dummy Encoding
# Create dummy variables for the Country column
dummy = pd.get_dummies(df, columns=['Country'], drop_first=True, prefix='DM')
print(dummy.columns)

Index(['SurveyDate', 'FormalEducation', 'ConvertedSalary', 'Hobby',
       'StackOverflowJobsRecommend', 'VersionControl', 'Age',
       'Years Experience', 'Gender', 'RawSalary', 'DM_India', 'DM_Ireland',
       'DM_Russia', 'DM_South Africa', 'DM_Spain', 'DM_Sweeden', 'DM_UK',
       'DM_USA', 'DM_Ukraine'],
      dtype='object')


In [15]:
#Dealing with uncommon variables(Masking)

# Create a series out of the Country column
countries = df['Country']

# Get the counts of each category
country_counts = countries.value_counts()

# Print the count values for each category
print(country_counts)

South Africa    166
USA             164
Spain           134
Sweeden         119
France          115
Russia           97
India            95
UK               95
Ukraine           9
Ireland           5
Name: Country, dtype: int64


In [16]:
# Create a mask for only categories that occur less than 10 times
mask = countries.isin(country_counts[country_counts < 10].index)
print(mask.head())

0    False
1    False
2    False
3    False
4    False
Name: Country, dtype: bool


In [23]:
# Label all masked countries as Others
countries[mask] = 'Others'

<ipython-input-23-df36b753888e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries[mask] = 'Others'


In [24]:
# Print the updated category counts
print(pd.value_counts(countries))

South Africa    166
USA             164
Spain           134
Sweeden         119
France          115
Russia           97
India            95
UK               95
Others           14
Name: Country, dtype: int64


In [27]:
#Binarizing Numeric variables

# Create the Paid_Job column filled with zeros
df['Paid_Job'] = 0

# Replace all the Paid_Job values where ConvertedSalary is > 0
df.loc[df['ConvertedSalary'] > 0, 'Paid_Job'] = 1

# Print the first five rows of the columns
print(df[['Paid_Job', 'ConvertedSalary']].head())

   Paid_Job  ConvertedSalary
0         0              NaN
1         1          70841.0
2         0              NaN
3         1          21426.0
4         1          41671.0


In [30]:
#Binning Numeric Variables

# Bin the continuous variable ConvertedSalary into 5 bins
df['equal_binned'] = pd.cut(df['ConvertedSalary'], 5)

# Print the first 5 rows of the equal_binned column
print(df[['equal_binned', 'ConvertedSalary']].tail())

              equal_binned  ConvertedSalary
994                    NaN              NaN
995    (-2000.0, 400000.0]          58746.0
996    (-2000.0, 400000.0]          55000.0
997                    NaN              NaN
998  (800000.0, 1200000.0]        1000000.0


In [34]:
# Import numpy
import numpy as np

# Specify the boundaries of the bins
bins = [-np.inf, 10000, 50000, 100000, 150000, np.inf]

# Bin labels
labels = ['Very low', 'Low', 'Medium', 'High', 'Very high']

# Bin the continuous variable ConvertedSalary using these boundaries
df['boundary_binned'] = pd.cut(df['ConvertedSalary'], 
                                         bins, labels = labels)

# Print the first 5 rows of the boundary_binned column
print(df[['boundary_binned', 'ConvertedSalary']].head())

  boundary_binned  ConvertedSalary
0             NaN              NaN
1          Medium          70841.0
2             NaN              NaN
3             Low          21426.0
4             Low          41671.0


In [35]:
#Missing values

#Sparsing your data

# Subset the DataFrame
sub_df = df[['Age', 'Gender']]

# Print the number of non-missing values
print(sub_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Age     999 non-null    int64 
 1   Gender  693 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB
None


In [36]:
# Print the top 10 entries of the DataFrame
print(sub_df.head(10))

   Age  Gender
0   21    Male
1   38    Male
2   45     NaN
3   46    Male
4   39    Male
5   39    Male
6   34    Male
7   24  Female
8   23    Male
9   36     NaN


In [37]:
# Print the locations of the missing values
print(sub_df.head(10).isnull())


     Age  Gender
0  False   False
1  False   False
2  False    True
3  False   False
4  False   False
5  False   False
6  False   False
7  False   False
8  False   False
9  False    True


In [38]:
# Print the locations of the non-missing values
print(sub_df.head(10).notnull())

    Age  Gender
0  True    True
1  True    True
2  True   False
3  True    True
4  True    True
5  True    True
6  True    True
7  True    True
8  True    True
9  True   False


In [39]:
#Dealing with missing values

#listwise deletion

# Print the number of rows and columns
print(df.shape)

# Create a new DataFrame dropping all incomplete rows
no_missing_values_rows = df.dropna()

# Print the shape of the new DataFrame
print(no_missing_values_rows.shape)

(999, 14)
(264, 14)


In [40]:
# Create a new DataFrame dropping all columns with incomplete rows
no_missing_values_cols = df.dropna(how='any', axis=1)

# Print the shape of the new DataFrame
print(no_missing_values_cols.shape)

(999, 8)


In [43]:
# Drop all rows where Gender is missing
no_gender = df.dropna(subset=['Gender'])

# Print the shape of the new DataFrame
print(no_gender.shape)

(693, 14)


In [45]:
#Replacing missing values with constants

# Print the count of occurrences
print(df['Gender'].value_counts())


Male                                                                         632
Female                                                                        53
Transgender                                                                    2
Female;Male                                                                    2
Non-binary. genderqueer. or gender non-conforming                              1
Female;Male;Transgender;Non-binary. genderqueer. or gender non-conforming      1
Male;Non-binary. genderqueer. or gender non-conforming                         1
Female;Transgender                                                             1
Name: Gender, dtype: int64


In [47]:
# Replace missing values
df['Gender'].fillna(value = 'Not Given', inplace = True)

# Print the count of each value
print(df['Gender'].value_counts())

Male                                                                         632
Not Given                                                                    306
Female                                                                        53
Transgender                                                                    2
Female;Male                                                                    2
Non-binary. genderqueer. or gender non-conforming                              1
Female;Male;Transgender;Non-binary. genderqueer. or gender non-conforming      1
Male;Non-binary. genderqueer. or gender non-conforming                         1
Female;Transgender                                                             1
Name: Gender, dtype: int64
